In [68]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
sess = tf.InteractiveSession()

<b>Loading the data</b>

In [69]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [70]:
mnist.train

In [71]:
mnist.test

In [72]:
mnist.validation

In [73]:
width = 28
height = 28
flat = width*height
output_classes = 10
X = tf.placeholder(tf.float32, shape=[None, flat])
Y = tf.placeholder(tf.float32, shape=[None, output_classes])

In [74]:
X_image = tf.reshape(X, [-1,28,28,1])

In [75]:
X_image

<tf.Tensor 'Reshape_3:0' shape=(?, 28, 28, 1) dtype=float32>

In [121]:
def weights_init(x,c,n):
    weight = tf.Variable(tf.truncated_normal([x,x,c,n], stddev=0.1))
    return weight

def bias_init(n):
    bias = tf.Variable(tf.constant(0.1, shape=[n]))
    return bias

def conv_layer(X_image, x, c, n):
    weight = weights_init(x,c,n)
    bias = bias_init(n)
    conv1 = tf.nn.conv2d(X_image, weight, strides=[1,1,1,1], padding="SAME") + bias
    return conv1

def max_pool(x, n):
    max_pool = tf.nn.max_pool(x, ksize=[1,n,n,1], strides=[1,2,2,1], padding="SAME")
    return max_pool

def fc_layer(X, input_size, output):
    w_fc = tf.Variable(tf.truncated_normal([input_size, output]))
    b_fc = tf.Variable(tf.constant(0.1, shape=[output]))
    fc_layer = tf.matmul(X, w_fc)+b_fc
    return fc_layer

In [122]:

conv_1 = conv_layer(X_image, 5, 1, 32)

In [123]:
conv_1_relu = tf.nn.relu(conv_1)

In [124]:
conv_1_pool = max_pool(conv_1_relu, 2)

In [125]:
conv_2 = conv_layer(conv_1_pool, 5, 32, 64)

In [126]:
conv_2_relu = tf.nn.relu(conv_2)

In [127]:
conv_2_pool = max_pool(conv_2_relu, 2)
conv_2_matrix = tf.reshape(conv_2_pool, [-1, 7*7*64])

In [128]:
fc_1 = fc_layer(conv_2_matrix, 7*7*64, 1024)
rfc_1 = tf.nn.relu(fc_1)
keep_prob=tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(rfc_1, keep_prob)

In [129]:
fc_2 = fc_layer(layer_drop, 1024, 10)
rfc_2 = tf.nn.relu(fc_2)

In [130]:
y_pred = tf.nn.softmax(rfc_2)

In [131]:
y_pred

<tf.Tensor 'Softmax_3:0' shape=(?, 10) dtype=float32>

In [132]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(y_pred), reduction_indices=[1]))

In [133]:
optimizer = tf.train.AdamOptimizer(1e-9).minimize(cross_entropy)

In [134]:
correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(y_pred,1))

In [135]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [136]:
for i in range(1100):
    batch = mnist.train.next_batch(50)
    if (i%100==0):
        acc = accuracy.eval(feed_dict={X:batch[0], Y:batch[1], keep_prob:1.0})
        print("For Epoch {}, accuracy = {}".format(i, acc*100))
    optimizer.run(feed_dict={X:batch[0], Y:batch[1], keep_prob:0.5})

For Epoch 0, accuracy = 14.000000059604645
For Epoch 100, accuracy = 5.999999865889549
For Epoch 200, accuracy = 7.999999821186066
For Epoch 300, accuracy = 0.0
For Epoch 400, accuracy = 14.000000059604645
For Epoch 500, accuracy = 5.999999865889549
For Epoch 600, accuracy = 15.999999642372131
For Epoch 700, accuracy = 21.99999988079071
For Epoch 800, accuracy = 5.999999865889549
For Epoch 900, accuracy = 7.999999821186066
For Epoch 1000, accuracy = 10.000000149011612


In [140]:
# Evaluation Process
cumulative_accuracy = 0.0
for i in range(mnist.test.images.shape[0]):
    batch = mnist.test.next_batch(50)
    cumulative_accuracy += accuracy.eval(feed_dict={X:batch[0], Y:batch[1], keep_prob:1.0})
    print(cumulative_accuracy)
print("Accuracy = {}",format(cumulative_accuracy/mnist.test.images.shape[0]))

0.1599999964237213
0.1599999964237213
0.3199999928474426
0.3999999910593033
0.4399999901652336
0.5999999865889549
0.6799999848008156
0.7799999862909317
0.8599999845027924
0.939999982714653
0.9999999813735485
1.139999981969595
1.2199999801814556
1.359999980777502
1.479999978095293
1.599999975413084
1.6999999769032001
1.8199999742209911
1.8799999728798866
2.039999969303608
2.0799999684095383
2.0999999679625034
2.2199999652802944
2.299999963492155
2.3599999621510506
2.3799999617040157
2.559999968856573
2.7399999760091305
2.819999974220991
2.9199999757111073
3.0199999772012234
3.099999975413084
3.2399999760091305
3.319999974220991
3.399999972432852
3.4799999706447124
3.5999999679625034
3.7199999652802944
3.8199999667704105
3.859999965876341
3.959999967366457
3.979999966919422
4.039999965578318
4.159999962896109
4.359999965876341
4.419999964535236
4.459999963641167
4.579999960958958
4.679999962449074
4.7599999606609344
4.81999995931983
4.959999959915876
5.099999960511923
5.199999962002039
5

43.47999977320433
43.51999977231026
43.55999977141619
43.63999976962805
43.69999976828694
43.779999766498804
43.859999764710665
43.87999976426363
43.939999762922525
44.07999976351857
44.21999976411462
44.319999765604734
44.53999976441264
44.739999767392874
44.83999976888299
44.899999767541885
44.979999765753746
45.09999976307154
45.19999976456165
45.29999976605177
45.41999976336956
45.49999976158142
45.65999975800514
45.69999975711107
45.739999756217
45.7999997548759
45.859999753534794
45.939999751746655
46.03999975323677
46.17999975383282
46.19999975338578
46.2999997548759
46.37999975308776
46.49999975040555
46.599999751895666
46.77999975904822
46.859999757260084
46.9599997587502
47.11999975517392
47.17999975383282
47.31999975442886
47.53999975323677
47.61999975144863
47.71999975293875
47.859999753534794
47.99999975413084
48.11999975144863
48.19999974966049
48.37999975681305
48.439999755471945
48.57999975606799
48.61999975517392
48.77999975159764
48.87999975308776
48.899999752640724
4

88.91999947652221
88.93999947607517
89.01999947428703
89.11999947577715
89.15999947488308
89.2599994763732
89.31999947503209
89.43999947234988
89.51999947056174
89.63999946787953
89.69999946653843
89.85999946296215
89.91999946162105
90.05999946221709
90.17999945953488
90.25999945774674
90.4399994648993
90.4999994635582
90.53999946266413
90.57999946177006
90.65999945998192
90.81999945640564
90.91999945789576
91.09999946504831
91.19999946653843
91.27999946475029
91.35999946296215
91.47999946027994
91.5599994584918
91.65999945998192
91.77999945729971
91.8399994559586
91.97999945655465
92.05999945476651
92.15999945625663
92.25999945774674
92.53999945893884
92.69999945536256
92.85999945178628
92.9599994532764
93.09999945387244
93.15999945253134
93.2399994507432
93.33999945223331
93.43999945372343
93.51999945193529
93.63999944925308
93.7399994507432
93.7999994494021
93.93999944999814
94.07999945059419
94.15999944880605
94.19999944791198
94.27999944612384
94.33999944478273
94.45999944210052
9

132.01999915763736
132.11999915912747
132.2199991606176
132.35999916121364
132.49999916180968
132.69999916478992
132.8199991621077
132.9399991594255
133.0599991567433
133.13999915495515
133.19999915361404
133.2799991518259
133.35999915003777
133.5199991464615
133.55999914556742
133.59999914467335
133.65999914333224
133.7999991439283
133.87999914214015
134.03999913856387
134.09999913722277
134.17999913543463
134.29999913275242
134.39999913424253
134.4799991324544
134.5799991339445
134.65999913215637
134.7599991336465
134.8599991351366
134.93999913334846
135.0799991339445
135.23999913036823
135.37999913096428
135.45999912917614
135.57999912649393
135.67999912798405
135.77999912947416
135.85999912768602
135.95999912917614
136.07999912649393
136.17999912798405
136.29999912530184
136.43999912589788
136.55999912321568
136.5999991223216
136.67999912053347
136.89999911934137
136.91999911889434
136.9999991171062
137.07999911531806
137.15999911352992
137.29999911412597
137.39999911561608
137.519

175.4799988232553
175.49999882280827
175.59999882429838
175.71999882161617
175.83999881893396
175.93999882042408
176.11999882757664
176.27999882400036
176.4199988245964
176.55999882519245
176.61999882385135
176.67999882251024
176.71999882161617
176.8199988231063
176.87999882176518
176.91999882087111
177.01999882236123
177.17999881878495
177.21999881789088
177.27999881654978
177.3199988156557
177.3799988143146
177.41999881342053
177.4999988116324
177.63999881222844
177.7199988104403
177.7799988090992
177.8799988105893
177.9399988092482
178.03999881073833
178.07999880984426
178.2199988104403
178.25999880954623
178.3399988077581
178.399998806417
178.53999880701303
178.6199988052249
178.8399988040328
178.91999880224466
178.97999880090356
179.05999879911542
179.15999880060554
179.39999879524112
179.49999879673123
179.5799987949431
179.65999879315495
179.77999879047275
179.83999878913164
179.95999878644943
180.0399987846613
180.15999878197908
180.2599987834692
180.33999878168106
180.45999877

...

753.6599946804345
753.6999946795404
753.7599946781993
753.919994674623
754.0399946719408
754.0599946714938
754.1999946720898
754.2999946735799
754.3599946722388
754.3799946717918
754.4799946732819
754.619994673878
754.6799946725368
754.8199946731329
754.8799946717918
755.0199946723878
755.119994673878
755.1999946720898
755.3399946726859
755.4999946691096
755.5599946677685
755.5999946668744
755.6799946650863
755.8199946656823
755.9399946630001
756.0599946603179
756.1799946576357
756.3199946582317
756.4599946588278
756.4999946579337
756.5799946561456
756.6199946552515
756.6999946534634
756.7999946549535
756.8799946531653
756.9999946504831
757.1199946478009
757.219994649291
757.2999946475029
757.3599946461618
757.4799946434796
757.5999946407974
757.7799946479499
757.8599946461618
757.9399946443737
758.0599946416914
758.1199946403503
758.1599946394563
758.2599946409464
758.3799946382642
758.459994636476
758.5599946379662
758.6999946385622
758.8799946457148
758.9599946439266
759.05999464541

799.0599942952394
799.1999942958355
799.3399942964315
799.4399942979217
799.5399942994118
799.5999942980707
799.7199942953885
799.7999942936003
799.8799942918122
799.9799942933023
800.139994289726
800.2399942912161
800.2799942903221
800.319994289428
800.3799942880869
800.5599942952394
800.6999942958355
800.7399942949414
800.8199942931533
800.9399942904711
801.0199942886829
801.159994289279
801.2599942907691
801.3999942913651
801.479994289577
801.5199942886829
801.6399942860007
801.6599942855537
801.7999942861497
801.9199942834675
802.0399942807853
802.1599942781031
802.2999942786992
802.419994276017
802.4599942751229
802.5799942724407
802.6399942710996
802.7199942693114
802.7799942679703
802.8999942652881
802.9999942667782
803.0999942682683
803.1999942697585
803.2999942712486
803.3799942694604
803.4199942685664
803.4799942672253
803.5399942658842
803.659994263202
803.7399942614138
803.8999942578375
803.9799942560494
804.0999942533672
804.1599942520261
804.2999942526221
804.439994253218

844.1199939846992
844.2199939861894
844.2799939848483
844.3199939839542
844.4199939854443
844.5199939869344
844.5999939851463
844.6999939866364
844.7599939852953
844.7999939844012
844.8999939858913
844.9799939841032
845.1199939846992
845.2599939852953
845.2799939848483
845.4799939878285
845.4999939873815
845.5599939860404
845.6799939833581
845.7799939848483
845.9199939854443
846.079993981868
846.2199939824641
846.279993981123
846.3599939793348
846.5399939864874
846.6599939838052
846.779993981123
846.919993981719
846.959993980825
847.0399939790368
847.1999939754605
847.2999939769506
847.4199939742684
847.5399939715862
847.6399939730763
847.6599939726293
847.7799939699471
847.9399939663708
847.9999939650297
848.0999939665198
848.2599939629436
848.3199939616024
848.4599939621985
848.4999939613044
848.5199939608574
848.5599939599633
848.6799939572811
848.7799939587712
848.8599939569831
848.9999939575791
849.1399939581752
849.2999939545989
849.399993956089
849.4999939575791
849.619993954896

889.8999936841428
889.9199936836958
889.9799936823547
890.1199936829507
890.2199936844409
890.3799936808646
890.5199936814606
890.5799936801195
890.6999936774373
890.7799936756492
890.8799936771393
890.9399936757982
891.0399936772883
891.1399936787784
891.2799936793745
891.3399936780334
891.4599936753511
891.5599936768413
891.5999936759472
891.6599936746061
891.7799936719239
891.8399936705828
891.9799936711788
892.0199936702847
892.1799936667085
892.2999936640263
892.3399936631322
892.4399936646223
892.5799936652184
892.6399936638772
892.7199936620891
892.799993660301
892.8599936589599
893.0199936553836
893.0599936544895
893.1399936527014
893.2599936500192
893.379993647337
893.4399936459959
893.5199936442077
893.5799936428666
893.6999936401844
893.8999936431646
894.0199936404824
894.1399936378002
894.259993635118
894.3599936366081
894.4799936339259
894.619993634522
894.6999936327338
894.899993635714
894.9999936372042
895.0399936363101
895.099993634969
895.119993634522
895.2399936318398

935.039993301034
935.1599932983518
935.2599932998419
935.4199932962656
935.5199932977557
935.6799932941794
935.8199932947755
935.9199932962656
936.0599932968616
936.2399933040142
936.2999933026731
936.3999933041632
936.4599933028221
936.5799933001399
936.6199932992458
936.7599932998419
936.859993301332
936.9799932986498
937.0399932973087
937.1199932955205
937.1999932937324
937.2799932919443
937.3399932906032
937.3799932897091
937.419993288815
937.4999932870269
937.599993288517
937.6799932867289
937.7399932853878
937.8399932868779
937.939993288368
938.1199932955205
938.1999932937324
938.2399932928383
938.3399932943285
938.4999932907522
938.5599932894111
938.7599932923913
938.8599932938814
938.9199932925403
939.0399932898581
939.1399932913482
939.2399932928383
939.2599932923913
939.3199932910502
939.3599932901561
939.439993288368
939.5199932865798
939.5999932847917
939.7399932853878
939.8399932868779
939.9199932850897
940.0999932922423
940.2199932895601
940.259993288666
940.3599932901561